In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import arviz as az
import bambi as bmb
import kulprit as kpt

import torch

/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
NUM_DRAWS, NUM_CHAINS = 50, 2

In [23]:
# load baseball data
df = bmb.load_data("batting").head(50)

In [24]:
# build model with a variate family not yet implemented
model = bmb.Model("p(H, AB) ~ 0 + playerID", df, family="binomial")
idata = model.fit(draws=NUM_DRAWS, chains=NUM_CHAINS)

Only 50 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [playerID]


Sampling 2 chains for 1_000 tune and 50 draw iterations (2_000 + 100 draws total) took 37 seconds.
/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.7/site-packages/pymc3/sampling.py:643: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn("The number of samples is too small to check convergence reliably.")


In [ ]:
idata

In [16]:
idata.to_json("../tests/data/binomial.json")

'../tests/data/regression.json'

In [14]:
az.from_json("regression.json") == idata

True

In [8]:
az.load_arviz_data("regression1d")

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In [3]:
# define model data
data = pd.DataFrame(
    {
        "y": np.random.normal(size=50),
        "g": np.random.choice(["Yes", "No"], size=50),
        "x1": np.random.normal(size=50),
        "x2": np.random.normal(size=50),
    }
)
# define and fit model with MCMC
model = bmb.Model("y ~ x1 + x2", data, family="gaussian")
num_draws, num_chains = 2000, 2
idata = model.fit(draws=num_draws, chains=num_chains)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [y_sigma, x2, x1, Intercept]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 30 seconds.


In [4]:
ref_model = kpt.ReferenceModel(model, idata)

In [12]:
sub_model = ref_model.project(terms=["x1"])
az.compare(dict(sub=sub_model.idata, full=idata))

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
full,0,-72.53915,3.652121,0.00000,1.0,5.060953,0.000000,False,log
sub,1,-73.37694,3.822604,0.83779,0.0,4.434442,0.778136,False,log


---

In [69]:
from abc import ABC, abstractmethod

import torch
import torch.nn as nn

class Loss(nn.Module, ABC):
    """Base loss class."""

    @abstractmethod
    def forward(self):
        pass


class KullbackLeiblerLoss(Loss):
    """Kullback-Leibler (KL) divergence loss module.

    This class computes some KL divergence loss for observations seen from the
    the reference model variate's family. The KL divergence is the originally
    motivated loss function by Goutis and Robert (1998).
    """

    def __init__(self, ref_model):
        """Loss module constructor.

        Args:
            ref_model (kulprit.data.ModelData): Reference model dataclass object
        """

        super().__init__()

        # define all available KL divergence loss classes
        self.family_dict = {
            "gaussian": GaussianKullbackLeiblerLoss,
        }

        # log family name
        self.family = ref_model.family

        if self.family not in self.family_dict:
            raise NotImplementedError(
                f"The {self.family} class has not yet been implemented."
            )

    def factory_method(self) -> Loss:
        """Choose the appropriate divergence class given the model."""

        # return appropriate divergence class given model variate family
        div_class = self.family_dict[self.family]
        return div_class()

    def forward(self, P: torch.tensor, Q: torch.tensor) -> torch.tensor:
        """Forward method in learning loop.

        This method computes the Kullback-Leibler divergence between the
        reference model variate draws ``P``and the restricted model's variate
        draws ``Q``. This is done using the two samples' respective sufficient
        sample statistics and a divergence equation found in the ``Family``
        class.

        Args:
            P (torch.tensor): Tensor of the reference model posterior MCMC
                draws
            Q (torch.tensor): Tensor of the restricted model posterior MCMC
                draws

        Returns:
            torch.tensor: Tensor of shape () containing sample KL divergence
        """

        div_class = self.factory_method()
        divs = div_class.forward(P, Q)
        return divs


class GaussianKullbackLeiblerLoss(Loss):
    """Gaussian empirical KL divergence class."""

    def forward(self, P: torch.tensor, Q: torch.tensor) -> torch.tensor:
        """Kullback-Leibler divergence between two Gaussians.

        Args:
            P (torch.tensor): Tensor of reference model posterior draws
            Q (torch.tensor): Tensor of restricted model posterior draws

        Returns:
            torch.tensor: Tensor of shape () containing sample KL divergence
        """

        # compute Wasserstein distance as a KL divergence surrogate
        div = torch.mean((P - Q) ** 2)

        assert div.shape == (), f"Expected data dimensions {()}, received {div.shape}."
        return div

In [70]:
class RefModel:
    def __init__(self, family):
        self.family = family

ref_model = RefModel("gaussian")
loss = KullbackLeiblerLoss(ref_model)

In [71]:
loss.forward(p, q)

tensor(0.)

In [61]:
p = torch.tensor([1.0, 1.0])
q = torch.tensor([1.0, 1.0])
loss.forward(p, q)

TypeError: forward() takes 1 positional argument but 2 were given

---

In [27]:
from dataclasses import dataclass, field

In [36]:
@dataclass
class Rectangle:
    height: float
    width: float

@dataclass
class Square(Rectangle):
    side: float

    def __post_init__(self):
        super().__init__(self.side, self.side)

In [37]:
square = Square(side=1.0)

TypeError: __init__() missing 2 required positional arguments: 'height' and 'width'